In [1]:
#Librerias necesarias
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   8449      0 --:--:-- --:--:-- --:--:--  8449
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


# Código revisado

In [0]:
import numpy as np
import cupy as cp
import solver.extraer_datos_yahoo as extrae
import solver.funciones_auxiliares as aux
import solver.line_search as line
import solver.modelo_markowitz as mkv
import solver.utils as utils
import solver.optimizacion_numerica as opt
#from utils import inc_index, dec_index, compute_error, norm_residual, condicion_cupy
#from line_search import line_search_by_backtracking, line_search_for_residual_by_backtracking

## Código módulos

### Extraer datos


In [0]:
import fix_yahoo_finance as yf

def extraer_datos_yahoo(stocks, start='2015-01-01', end='2020-04-30'):
  '''
  Funcion para extraer precios al cierre de las acciones mediante yahoo finance de 2015-01-01 a 2020-04-30

  params: stocks    lista de acciones de las cuales se desea obtener el precio
          start     fecha inicial
          end       fecha final

  return: base      Dataframe de precios por acción (columnas) y día (filas)
  '''
  df_c = yf.download(stocks, start=start, end=end).Close
  base = df_c['AAPL'].dropna().to_frame()
  for i in range(0,50):
      base = base.join(df_c.iloc[:,i].to_frame(), lsuffix='_caller', rsuffix='_other')
  base = base.drop(columns=['AAPL_caller'])
  base = base.rename(columns={"AAPL_other": "AAPL"})
  base = base.fillna(method='ffill')
  base = base.fillna(method='bfill')
  return base


### Funciones auxiliares

In [0]:
import cupy as cp

def calcular_rendimiento_vector(x):
  """
  Función para calcular el rendimiento esperado

  params:
      x     vector de precios
  
  return:
      r_est rendimiento esperado diario
  """

  # Definimos precios iniciales y finales como arreglo alojado en la gpu
  x_o = cp.asarray(x)
  x_f = x_o[1:]

  # Calculamos los rendimientos diarios
  r = cp.log(x_f/x_o[:-1])

  return r

def calcular_rendimiento(X):
  """
  Función para calcular el rendimiento esperado para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
             n el número de acciones
  
  return:
      r_est rvector de rendimientos esperados
  """
  m,n = X.shape
  r_est = cp.zeros(n)
  X = cp.asarray(X)

  for i in range(n):
    r_est[i] = calcular_rendimiento_vector(X[:,i]).mean()

  return 264*r_est

def calcular_varianza(X):

  """
  Función para calcular el la matriz de varianzas y covarianzas para un conjunto de acciones

  params:
      X      matriz mxn de precios, donde:
             m es el número de observaciones y
               n el número de acciones
  
  return:
      S  matriz de varianzas y covarianzas
  """
  m,n=X.shape
  X = cp.asarray(X)

  X_m = cp.zeros((m-1,n))

  for i in range(n):
    X_m[:,i] = calcular_rendimiento_vector(X[:,i]) - calcular_rendimiento_vector(X[:,i]).mean()

  S = (cp.transpose(X_m)@X_m)/(m-2)

  return S




### Modelo Markovitz

In [0]:
import cupy as cp
import pandas as pd

def formar_vectores(mu, Sigma):
  '''
  Calcula las cantidades u = \Sigma^{-1}  \mu y v := \Sigma^{-1} \cdot 1 del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    u (cupy array, escalar): vector dado por \cdot Sigma^-1 \cdot mu (dimension n)
    v (cupy array, escalar): vector dado por Sigma^-1 \cdot 1 (dimension n)
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  # Obtiene vectores de interes
  u = cp.linalg.solve(Sigma, mu)
  u = u.transpose() # correcion de expresion de array
  v = cp.linalg.solve(Sigma, ones_vector)

  return u , v

def formar_abc(mu, Sigma):
  '''
  Calcula las cantidades A, B y C del diagrama de flujo del problema de Markowitz

  Args:
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu
  '''

  # Vector auxiliar con entradas igual a 1
  n = Sigma.shape[0]
  ones_vector = cp.ones(n)

  # Formamos vector \cdot Sigma^-1 mu y Sigm^-1 1
  # Nota: 
  #   1) u= Sigma^-1 \cdot mu se obtiene resolviendo  Sigma u = mu
  #   2) v= Sigma^-1 \cdot 1 se obtiene resolviendo  Sigma v = 1

  u, v = formar_vectores(mu, Sigma)

  # Obtiene escalares de interes
  A = mu.transpose()@u
  B = ones_vector.transpose()@v
  C = ones_vector.transpose()@u

  return A, B, C


def delta(A,B,C):
  '''
  Calcula las cantidad Delta = AB-C^2 del diagrama de flujo del problema de Markowitz

  Args:
    A (cupy array, escalar): escalar dado por mu^t \cdot Sigma^-1 \cdot mu
    B (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot 1
    C (cupy array, escalar): escalar dado por 1^t \cdot Sigma^-1 \cdot mu

  Return:
    Delta (cupy array, escalar): escalar dado \mu^t \cdot \Sigma^{-1} \cdot \mu
  '''
  Delta = A*B-C**2

  return Delta


def formar_omegas(r, mu, Sigma):
  '''
  Calcula las cantidades w_o y w_ del problema de Markowitz

  Args:
    r (escalar) : retorno esperado por el inversionista
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    w_0 (cupy escalar, matriz): escalar dado por 
          w_0 = \frac{1}{\Delta} (B \Sigma^{-1} \hat{\mu}- C\Sigma^{-1} 1) 
    w_1 (cupy escalar, vector): escalar dado por 
         w_1 = \frac{1}{\Delta} (C \Sigma^{-1} \hat{\mu}- A\Sigma^{-1} 1)
  '''
  # Obtenemos u = Sigma^{-1} \hat{\mu}, v = \Sigma^{-1} 1
  u, v = formar_vectores(mu, Sigma)
  # Escalares relevantes
  A, B, C = formar_abc(mu, Sigma)
  Delta = delta(A,B,C)
  # Formamos w_0 y w_1
  w_0 = (1/Delta)*(r*B-C)
  w_1 = (1/Delta)*(A-C*r)

  return w_0, w_1

def markowitz(r, mu, Sigma):
  '''
  Calcula las cantidades w_o y w_ del problema de Markowitz

  Args:
    r (escalar) : retorno esperado por el inversionista
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)

  Return:
    w (cupy array, vector): vector de pesos que minimizan la varianza del portfolio
  '''
  # Obtenemos u = Sigma^{-1} \hat{\mu}, v = \Sigma^{-1} 1
  u, v = formar_vectores(mu, Sigma)

  # Formamos w_0 y w_1
  w_0, w_1 = formar_omegas(r, mu, Sigma)

  return w_0*u+w_1*v

def markowitz_df(r, mu, Sigma, stocks):
  '''
  Crea data frame de la función markowitz relacionando los pesos con las acciones

  Args:
    r (escalar) : retorno esperado por el inversionista
    mu (cupy array, vector): valores medios esperados de activos (dimension n)
    Sigma (cupy array, matriz): matriz de covarianzas asociada a activos (dimension n x n)
    stocks (lista): Códigos bursátiles de las acciones a evaluar

  Return:
    df (DataFrame): Data Frame con acciones como índices y los pesos ordenados correspondientes
  '''

  aux = markowitz(r, mu, Sigma)
  df =pd.DataFrame( cp.asnumpy(aux), columns = ['pesos'])
  df["stocks"]=stocks
  df = df.sort_values(by=['pesos'], ascending=False)
  return df




### Optimización numérica

In [0]:
import cupy as cp
import numpy as np
from solver.utils import inc_index
from solver.utils import dec_index


def gfo_cp_mark(Sigma,x):
    '''
    gradiente de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma y vector x
    output: producto matriz-vector Sigma*x
    '''
    first_block = Sigma@x
    return first_block


def Hfo_cp_mark(Sigma):
    '''
    Hessiana de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma
    output: matriz Sigma
    '''
    first_block = Sigma 
    return first_block


def gradient_approximation(f,x,h=1e-8):
    '''
    Numerical approximation of gradient for function f using forward differences.
    Args:
        f (lambda expression): definition of function f.
        x (array): numpy array that holds values where gradient will be computed.
        h (float): step size for forward differences, tipically h=1e-8
    Returns:
        gf (array): numerical approximation to gradient of f.
    '''
    n = x.size
    gf = cp.zeros(n)
    f_x = f(x)
    for i in np.arange(n):
        inc_index(x,i,h)
        gf[i] = f(x) - f_x
        dec_index(x,i,h)
    return gf/h



def Hessian_approximation(f,x,h=1e-6):
    '''
    Numerical approximation of Hessian for function f using forward differences.
    Args:
        f (lambda expression): definition of function f.
        x (array): numpy array that holds values where Hessian will be computed.
        h (float): step size for forward differences, tipically h=1e-6
    Returns:
        Hf (array): numerical approximation to Hessian of f.
    '''
    n = x.size
    Hf = cp.zeros((n,n))
    f_x = f(x)
    for i in np.arange(n):
        inc_index(x,i,h)
        f_x_inc_in_i = f(x)
        for j in np.arange(i,n):
            inc_index(x,j,h)
            f_x_inc_in_i_j = f(x)
            dec_index(x,i,h)
            f_x_inc_in_j = f(x)
            dif = f_x_inc_in_i_j-f_x_inc_in_i-f_x_inc_in_j+f_x
            Hf[i,j] = dif
            if j != i:
                Hf[j,i] = dif
            dec_index(x,j,h)
            inc_index(x,i,h)
        dec_index(x,i,h)
    return Hf/h**2

import solver



def Newtons_method_feasible_init_point(f, A, x_0, tol, 
                                       tol_backtracking, x_ast=None, p_ast=None, maxiter=30,
                                       gf_symbolic = None,
                                       Hf_symbolic = None,
                                       Sigma = None):
    '''
    Newton's method to numerically approximate solution of min f subject to Ax = b.
    IMPORTANT: this implementation requires that initial point x_0, satisfies: Ax_0 = b
    Args:
        f (fun): definition of function f as lambda expression or function definition.
        A (numpy ndarray): 2d numpy array of shape (m,n) defines system of constraints Ax=b.
        x_0 (numpy ndarray): initial point for Newton's method. Must satisfy: Ax_0 = b
        tol (float): tolerance that will halt method. Controls stopping criteria.
        tol_backtracking (float): tolerance that will halt method. Controls value of line search by backtracking.
        x_ast (numpy ndarray): solution of min f, now it's required that user knows the solution...
        p_ast (float): value of f(x_ast), now it's required that user knows the solution...
        maxiter (int): maximum number of iterations
        gf_symbolic (fun): definition of gradient of f. If given, no approximation is
                                     performed via finite differences.
        Hf_symbolic (fun): definition of Hessian of f. If given, no approximation is
                                     performed via fi
                                     nite differences.
    Returns:
        x (numpy ndarray): numpy array, approximation of x_ast.
        iteration (int): number of iterations.
        Err_plot (numpy ndarray): numpy array of absolute error between p_ast and f(x) with x approximation
                          of x_ast. Useful for plotting.
        x_plot (numpy ndarray): numpy array that containts in columns vector of approximations. Last column
                        contains x, approximation of solution. Useful for plotting.
    '''
    iteration = 0
        
    x = x_0
    
    feval = f(x)
    
    if gf_symbolic:
        gfeval = gf_symbolic(x,Sigma)
    else:
        gfeval = gradient_approximation(f,x)

    if Hf_symbolic:
        Hfeval = Hf_symbolic(x,Sigma)
    else:
        Hfeval = Hessian_approximation(f,x)
    
    normgf = np.linalg.norm(gfeval)
    condHf= solver.utils.condicion_cupy(Hfeval)
    
    Err_plot_aux = np.zeros(maxiter)
    Err_plot_aux[iteration]= solver.utils.compute_error(p_ast,feval)
    
    Err = solver.utils.compute_error(x_ast,x)
    
        
    if(A.ndim == 1):
        p = 1
        n = x.size
        zero_matrix = cp.zeros(p)
        first_stack = cp.column_stack((Hfeval, A.T))
        second_stack = cp.row_stack((A.reshape(1,n).T,zero_matrix)).reshape(1,n+1)[0]
    else:
        p,n = A.shape
        zero_matrix = cp.zeros((p,p))
        first_stack = np.column_stack((Hfeval, A.T))
        second_stack = np.column_stack((A,zero_matrix))
        
    x_plot = cp.zeros((n,maxiter))
    x_plot[:,iteration] = x
    
    system_matrix = cp.vstack((first_stack,second_stack))
    zero_vector = cp.zeros(p)
    rhs = cp.vstack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]

    #Newton's direction and Newton's decrement
    dir_desc = cp.linalg.solve(system_matrix, -rhs)
    dir_Newton = dir_desc[0:n]
    dec_Newton = -gfeval.dot(dir_Newton)
    w_dual_variable_estimation = dir_desc[n:(n+p)]


    print('I\tNormgf \tNewton Decrement\tError x_ast\tError p_ast\tline search\tCondHf')
    print('{}\t{}\t{}\t{}\t{}\t{}\t\t{}'.format(iteration,
                                                cp.around(normgf,4),
                                                cp.around(dec_Newton,4),
                                                cp.around(Err,4),
                                                cp.around(Err_plot_aux[iteration],4),
                                                "---",
                                                cp.around(condHf,4)))
    stopping_criteria = dec_Newton/2
    iteration+=1
    while(stopping_criteria>tol and iteration < maxiter):
        der_direct = -dec_Newton
        t = solver.line_search.line_search_by_backtracking(f,dir_Newton,x,der_direct)
        x = x + t*dir_Newton
        feval = f(x)
        
        
        if gf_symbolic:
            gfeval = gf_symbolic(x,Sigma)
        else:
            gfeval = gradient_approximation(f,x)
        
        if Hf_symbolic: 
            Hfeval = Hf_symbolic(x,Sigma)
        else:
            Hfeval = Hessian_approximation(f,x)
        if(A.ndim == 1):
            first_stack = cp.column_stack((Hfeval, A.T))
        else:
            first_stack = cp.column_stack((Hfeval, A.T))

        system_matrix = cp.vstack((first_stack,second_stack))
        rhs = cp.vstack((gfeval.reshape(n,1), zero_vector.reshape(p,1))).T[0]
        #Newton's direction and Newton's decrement
        dir_desc = cp.linalg.solve(system_matrix, -rhs)
        dir_Newton = dir_desc[0:n]
        dec_Newton = -gfeval.dot(dir_Newton)
        w_dual_variable_estimation = dir_desc[n:(n+p)]
        
        Err_plot_aux[iteration]= solver.utils.compute_error(p_ast,feval)
        x_plot[:,iteration] = x
        Err = solver.utils.compute_error(x_ast,x)
        print('{}\t{}\t{}\t{}\t{}\t{}\t{}'.format(iteration,
                                                  cp.around(normgf,4),
                                                  cp.around(dec_Newton,4),
                                                  cp.around(Err,4),
                                                  cp.around(Err_plot_aux[iteration],4),
                                                  cp.around(t,4),
                                                  cp.around(condHf,4)))
        stopping_criteria = dec_Newton/2
        if t<tol_backtracking: #if t is less than tol_backtracking then we need to check the reason
            iter_salida=iteration
            iteration = maxiter - 1
        iteration+=1
    print('{} {}'.format("Error of x with respect to x_ast:",Err))
    print('{} {}'.format("Approximate solution:", x))
    cond = Err_plot_aux > np.finfo(float).eps*10**(-2)
    Err_plot = Err_plot_aux[cond]
    
    if iteration == maxiter and t < tol_backtracking:
        print("Backtracking value less than tol_backtracking, check approximation")
        iteration=iter_salida
    else:
        if iteration == maxiter:
            print("Reached maximum of iterations, check approximation")
    x_plot = x_plot[:,:iteration]
    return [x,iteration,Err_plot,x_plot]


### Utils

In [0]:
import cupy as cp

def inc_index(vec,index,h):
    '''
    Funcion auxiliar para calculo gradiente y hessiano.
    Args:
        vec (array): array de cupy.
        index (int): indice.
        h (float):   cantidad del vec[index] que sera incrementada.
    Returns:
        vec (array): array vec de cupy con vec[index] incrementado con h.
    '''
    vec[index] +=h
    return vec

def dec_index(vec,index,h=1):
    '''
    Funcion auxiliar para calculo gradiente y hessiano.
    Args:
        vec (array): array de cupy.
        index (int): indice.
        h (float):   antidad del vec[index] que sera disminuida.
    Returns:
        vec (array): array vec de cupy con vec[index] disminuida con h.
    '''
    vec[index] -=h
    return vec

def compute_error(x_obj,x_approx):
    '''
    Error absoluto o relativo entre x_obj y x_approx.
    '''
    if cp.linalg.norm(x_obj) > cp.nextafter(0,1):
        Err=cp.linalg.norm(x_obj-x_approx)/cp.linalg.norm(x_obj)
    else:
        Err=cp.linalg.norm(x_obj-x_approx)
    return Err

def norm_residual(feas_primal, feas_dual):
    '''
    Calcula la norma de residuos para el método de punto inicial no factible de Newton
    '''
    return cp.sqrt(cp.linalg.norm(feas_primal)**2 +\
                   cp.linalg.norm(feas_dual)**2
                   )

def condicion_cupy(A):
    '''
    Función que calcula la condición de una matriz para matrices en Cupy
    Args:
      A:  Matriz de cupy
    Return:
      c: Condición de una matriz
    '''
    A_inv = cp.linalg.inv(A)
    NAi = cp.linalg.norm(A_inv)
    NA = cp.linalg.norm(A)
    return NAi*NA

def feasible_markowitz(r,mu):
  '''
  Funcion que devuelve un punto factible del problema de Markowitz, resolviendo
  las ecuaciones normales asociadas
  Args:
     r (float): real positivo, que modela rendimiento esperado por inversionista
     mu (aray): array de cupy que contiene los valores medios de los rendimientos
          de los activos en el portafolio
  Return:
     w_star (float): punto feasible del problema de Markowitz
  '''
  # matriz del sistema para ecuaciones normales
  M = cp.ones((2,mu.shape[0]))

  for i in range(mu.shape[0]):
    M[0,i]=mu[i]

  # lado derecho de sistema
  b= cp.array([r,1])

  # Resuelve ecuaciones normales
  w_star = cp.linalg.solve(M.transpose()@M,M.transpose()@b)
  
  return w_star

### Punto factible

In [0]:
import cupy as cp
def feasible_markowitz(r,mu):
  '''
  Funcion que devuelve un punto factible del problema de Markowitz, resolviendo
  las ecuaciones normales asociadas
  Args:
     r (float): real positivo, que modela rendimiento esperado por inversionista
     mu (aray): array de cupy que contiene los valores medios de los rendimientos
          de los activos en el portafolio
  Return:
     w_star (float): punto feasible del problema de Markowitz
  '''
  # matriz del sistema para ecuaciones normales
  M = cp.ones((2,mu.shape[0]))

  for i in range(mu.shape[0]):
    M[0,i]=mu[i,0]

  # lado derecho de sistema
  b= cp.array([r,1])

  # Resuelve ecuaciones normales
  w_star = cp.linalg.solve(M.transpose()@M,M.transpose()@b)
  
  return w_star

### Line search

In [0]:
from solver.utils import norm_residual

def line_search_by_backtracking(f,dir_desc,x,
                                der_direct, alpha=.15, beta=.5):
    '''
    Busqueda de linea que disminuye suficientemente f restringida a un rayo en
    la dirección dir_desc.
    Args:
        alpha (float): parametro en busqueda de linea con backtracking, tipicamente .15
        beta (float): parametro en busqueda de linea con backtracking, tipicamente .5
        f (lambda expression): definicion de funcion f.
        dir_desc (array): direccion de descenso.
        x (array): array de cupy que contiene valores donde se realizara la busqueda de linea
        der_direct (float): derivada direccional de f.
    Returns:
        t (float): numero positivo para el tamano de pasos a lo largo de dir_desc que
                   disminuye suficientemente f.
    '''
    t=1
    if alpha > 1/2:
        print('alpha menor o igual que 1/2')
        t=-1
    if beta>1:
        print('beta debe ser menor a 1')
        t=-1;
    if t!=-1:
        eval1 = f(x+t*dir_desc)
        eval2 = f(x) + alpha*t*der_direct
        while eval1 > eval2:
            t=beta*t
            eval1=f(x+t*dir_desc)
            eval2=f(x)+alpha*t*der_direct
    else:
        t=-1
    return t

def line_search_for_residual_by_backtracking(r_primal, r_dual,dir_desc_primal,dir_desc_dual,x, nu,
                                             norm_residual_eval,
                                             alpha=.15, beta=.5):
    '''
    Busqueda de linea que disminuye suficientemente el residual para el metodo
    de punto inicial inviable de Newton restringido a un rayo en la direccion dir_desc.
    Args:
        r_primal (fun): definicion de residual primal como funcion definida o expresion lambda.
        r_dual (fun): definicion de residual dual como funcion definida o expresion lambda.
        dir_desc_primal (array): direccion de desceso para variable primal.
        dir_desc_dual (array): direccion de descenso para variable dual.
        x (array): array de cupy que contiene valores donde se realizara la busqueda de linea.
        nu (array): array de cupy que contiene valores donde se realizara la busqueda de linea.
        norm_residual_eval (float): norma de residuos que tiene evaluaciones r_primal y 
        r_dual en x y nu
        alpha (float): parametro de busqueda de linea con backtracking, tipicamente .15
        beta (float): parametro de busqueda de linea con backtracking, tipicamente .5

    Returns:
        t (float): numero positivo para el tamano de pasos a lo largo de dir_desc que 
        disminuye suficientemente f.
    '''
    t=1
    if alpha > 1/2:
        print('alpha debe ser menor o igual que 1/2')
        t=-1
    if beta>1:
        print('beta debe ser menor que uno 1')
        t=-1;
    if t!=-1:
        feas_primal = r_primal(x + t*dir_desc_primal)
        feas_dual = r_dual(nu + t*dir_desc_dual )
        eval1 = norm_residual(feas_primal, feas_dual)
        eval2 = (1-alpha*t)*norm_residual_eval
        while eval1 > eval2:
            t=beta*t
            feas_primal = r_primal(x + t*dir_desc_primal)
            feas_dual = r_dual(nu + t*dir_desc_dual )
            eval1 = norm_residual(feas_primal, feas_dual)
            eval2 = (1-alpha*t)*norm_residual_eval
    return t


## Cargamos la información y calculamos los parámetros

Primero cargamos los precios de las acciones que se han escogido para esta implementación

In [0]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

In [4]:
datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


Usando las fucniones auxiliares definidas previamente calculamos los rendimientos esperados y la matriz de varianza y covarianza para las acciones

In [0]:
mu = aux.calcular_rendimiento(datos)

In [0]:
S = aux.calcular_varianza(datos)

Calcularemos cuál fue el rendimiento máximo obtenido (de entre las medias de los datos históricos) y lo usaremos como el rendimiento deseado en el problema de optimización

In [7]:
r=max(mu).item()
r

0.4022108787760788

## Resolvemos con el Método de Newton usando diferencias finitas

La función a optimizar será la siguiente:

In [0]:
fo = lambda w: w@S@w

Usamos la solución analítica para tener la solución con la cual comparar.

In [0]:
w_ast = mkv.markowitz(r,mu,S)

La matriz $A$ de restricciones está dada por:

In [10]:
n = mu.shape[0]
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
A

array([[ 0.13254293,  0.02782955,  0.25778345, -0.01787111, -0.02008539,
        -0.02286483,  0.19078421,  0.05743478,  0.17062275,  0.40221088,
         0.13618598,  0.02364206,  0.0476638 ,  0.13217291, -0.09666644,
        -0.03426582,  0.15880965,  0.17870289,  0.18557566,  0.15042008,
         0.08379133,  0.07154137,  0.08843336,  0.02202303,  0.07353272,
         0.13731909,  0.23605478,  0.19942876,  0.06806557,  0.26292508,
         0.07357148,  0.06803817,  0.03867589,  0.05090149,  0.14401416,
        -0.09339291,  0.23072551,  0.05036842,  0.05587177, -0.01264577,
        -0.01474117,  0.20678446,  0.06274014, -0.02729424,  0.1990038 ,
         0.07054765,  0.06563628,  0.04203765,  0.0717407 , -0.13227261],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.

Por otro lado el vector $b$ está dado por:

In [11]:
b = cp.array([r,1])
b

array([0.40221088, 1.        ])

Definimos el punto inicial con la función definida anteriormente:

In [0]:
w_0 = utils.feasible_markowitz(r,mu)

Definimos los parámetros para el método de Newton de la siguiente manera:

In [0]:
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [15]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)



I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6964	---		11162.699
1	0.0019	0.0	0.0334	0.0002	1	11162.699
2	0.0019	0.0	0.0005	0.0	1	11162.699
Error of x with respect to x_ast: 0.00047875631346943525
Approximate solution: [ 1.58456068e-01 -2.08709832e-02  1.58048926e-01 -9.28997264e-02
  3.17031053e-02  5.10786741e-02  8.71853168e-02  1.26296186e-02
  3.72848119e-02  2.86001357e-01 -5.99024985e-03  2.24553601e-03
  2.03074487e-01  9.46087178e-02  2.28786290e-02  1.49784235e-02
  7.61265302e-03  2.96680495e-02  5.76467623e-02  1.98108419e-01
  1.19921167e-01  1.27865977e-01  1.41309880e-01  1.36173882e-02
  8.83887534e-02  1.50475380e-01  1.69271249e-01  7.72165387e-02
  8.09183143e-02  8.24853959e-02  1.92195287e-01 -2.39896300e-02
  2.64386780e-02  7.69729393e-02  2.20710514e-02 -1.08208343e-01
  1.64030583e-01  1.76988175e-02  6.16368407e-02 -1.08217542e-01
 -5.03608169e-02  1.38841035e-01  1.03558964e-01 -4.28149790e-02
  1.07055969e-02

La solución dada por el Método de Newton usando las diferenciación numérica es muy similar a la del método cerrado. Siendo el error de:

In [16]:
Err_plot[-1]

1.073648151492267e-08

In [17]:
w

array([ 1.58456068e-01, -2.08709832e-02,  1.58048926e-01, -9.28997264e-02,
        3.17031053e-02,  5.10786741e-02,  8.71853168e-02,  1.26296186e-02,
        3.72848119e-02,  2.86001357e-01, -5.99024985e-03,  2.24553601e-03,
        2.03074487e-01,  9.46087178e-02,  2.28786290e-02,  1.49784235e-02,
        7.61265302e-03,  2.96680495e-02,  5.76467623e-02,  1.98108419e-01,
        1.19921167e-01,  1.27865977e-01,  1.41309880e-01,  1.36173882e-02,
        8.83887534e-02,  1.50475380e-01,  1.69271249e-01,  7.72165387e-02,
        8.09183143e-02,  8.24853959e-02,  1.92195287e-01, -2.39896300e-02,
        2.64386780e-02,  7.69729393e-02,  2.20710514e-02, -1.08208343e-01,
        1.64030583e-01,  1.76988175e-02,  6.16368407e-02, -1.08217542e-01,
       -5.03608169e-02,  1.38841035e-01,  1.03558964e-01, -4.28149790e-02,
        1.07055969e-02, -2.33399846e+00,  1.90628556e-01,  2.63317339e-01,
        1.20089563e-01, -1.05514115e-01])

In [18]:
w_ast

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

La varianza entre ambos métodos también es muy similar

In [19]:
w@S@w

array(9.45979256e-05)

In [20]:
w_ast@S@w_ast

array(9.45979246e-05)

In [21]:
utils.compute_error(w@S@w,w_ast@S@w_ast)

array(1.07364814e-08)

Además cumple con la restricción del rendimiento y la suma de pesos igual a 1

In [22]:
w@mu

array(0.40221088)

In [23]:
utils.compute_error(w@mu, r)

array(1.38015042e-16)

In [24]:
sum(w)

array(1.)

## Resolvemos con el Método de Newton usando funciones simbólicas

Ahora utilizaremos de nuevo el método de Newton usando las funciones simbólicas:

In [0]:
import inspect

In [26]:
lines = inspect.getsource(opt.gfo_cp_mark)
print(lines)

def gfo_cp_mark(Sigma,x):
    '''
    gradiente de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma y vector x
    output: producto matriz-vector Sigma*x
    '''
    first_block = Sigma@x
    return first_block



In [27]:
lines = inspect.getsource(opt.Hfo_cp_mark)
print(lines)

def Hfo_cp_mark(Sigma):
    '''
    Hessiana de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma
    output: matriz Sigma
    '''
    first_block = Sigma 
    return first_block



In [28]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.001	0.0004	1.1003	4.6964	---		10080.7985
1	0.001	0.0	0.0	0.0	1	10080.7985
Error of x with respect to x_ast: 6.4486015181734205e-12
Approximate solution: [ 1.58450459e-01 -2.08710279e-02  1.58051613e-01 -9.28970852e-02
  3.17093838e-02  5.10824537e-02  8.71800626e-02  1.26389547e-02
  3.72861469e-02  2.86000507e-01 -5.98810147e-03  2.24206232e-03
  2.03075836e-01  9.46030741e-02  2.28766788e-02  1.49919976e-02
  7.60706433e-03  2.96676402e-02  5.76521006e-02  1.98109863e-01
  1.19928144e-01  1.27869501e-01  1.41300419e-01  1.36285005e-02
  8.83904753e-02  1.50479914e-01  1.69293512e-01  7.72037012e-02
  8.09121352e-02  8.24658724e-02  1.92197879e-01 -2.40095431e-02
  2.64375219e-02  7.69647088e-02  2.20741648e-02 -1.08207562e-01
  1.64032748e-01  1.77020932e-02  6.16398087e-02 -1.08210745e-01
 -5.03462807e-02  1.38834320e-01  1.03567400e-01 -4.28198353e-02
  1.06872745e-02 -2.33314798e+00  1.89811544e-01  2.632964

La solución dada por el Método de Newton usando las funciones simbólicas es prácticamente igual al método cerrado método cerrado. Siendo el error de:


In [29]:
Err_plot[-1]

2.865290589556516e-16

In [30]:
w

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

In [31]:
w_ast

array([ 1.58450459e-01, -2.08710279e-02,  1.58051613e-01, -9.28970852e-02,
        3.17093838e-02,  5.10824537e-02,  8.71800626e-02,  1.26389547e-02,
        3.72861469e-02,  2.86000507e-01, -5.98810147e-03,  2.24206232e-03,
        2.03075836e-01,  9.46030741e-02,  2.28766788e-02,  1.49919976e-02,
        7.60706433e-03,  2.96676402e-02,  5.76521006e-02,  1.98109863e-01,
        1.19928144e-01,  1.27869501e-01,  1.41300419e-01,  1.36285005e-02,
        8.83904753e-02,  1.50479914e-01,  1.69293512e-01,  7.72037012e-02,
        8.09121352e-02,  8.24658724e-02,  1.92197879e-01, -2.40095431e-02,
        2.64375219e-02,  7.69647088e-02,  2.20741648e-02, -1.08207562e-01,
        1.64032748e-01,  1.77020932e-02,  6.16398087e-02, -1.08210745e-01,
       -5.03462807e-02,  1.38834320e-01,  1.03567400e-01, -4.28198353e-02,
        1.06872745e-02, -2.33314798e+00,  1.89811544e-01,  2.63296457e-01,
        1.20089773e-01, -1.05535600e-01])

La varianza entre ambos métodos también es prácticamente la misma

In [32]:
w@S@w

array(9.45979246e-05)

In [33]:
w_ast@S@w_ast

array(9.45979246e-05)

In [34]:
utils.compute_error(w@S@w,w_ast@S@w_ast)

array(2.86529059e-16)

Y de igual manera cumple con la restricción del rendimiento y la suma de pesos igual a 1

In [35]:
w@mu

array(0.40221088)

In [36]:
utils.compute_error(w@mu, r)

array(0.)

In [37]:
sum(w)

array(1.)

# Resultados




**1. Documentacion**

La Documentación de las funciones expresa de manera clara, consica y breve lo que hace cada una. De igual forma se explican de manera clara y concisa los argumentos de entrada y salida. La documentación es completa.


**2. Cumplimiento de objetivos del código**

Las funciones cumplen con sus objetivos.

**3. Test 1**

Revisaremos las función con distintas iteraciones y tolerancias.


## Método de Newton usando diferencias finitas

In [38]:
stocks = ['COP','AMT','LIN','LMT','AMZN','WMT','JNJ','VTI','MSFT','GOOG','XOM','CCI','BHP.AX','UNP',
'BABA','NSRGY','RHHBY','VOO','AAPL','FB','CVX','PLD','RIO.L','HON','HD','PG','UNH','BRK-A','V','0700.HK',
'RDSA.AS','0688.HK','AI.PA','RTX','MC.PA','KO','PFE','JPM','005930.KS','VZ','RELIANCE.NS','DLR','2010.SR',
'UPS','7203.T','PEP','MRK','1398.HK','MA','T']

datos = extrae.extraer_datos_yahoo(stocks)

[*********************100%***********************]  50 of 50 downloaded


In [39]:
mu = aux.calcular_rendimiento(datos)
S = aux.calcular_varianza(datos)
r=max(mu).item()
r

0.4022108787760788

In [0]:
fo = lambda w: w@S@w

In [0]:

w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)

In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [43]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6965	---		9481.6958
1	0.0019	0.0	0.1041	0.0005	1	9481.6958
2	0.0019	0.0	0.0028	0.0	1	9481.6958
Error of x with respect to x_ast: 0.002754207971876388
Approximate solution: [ 1.58429838e-01 -2.08662572e-02  1.58036015e-01 -9.28827511e-02
  3.16796179e-02  5.10933254e-02  8.72794066e-02  1.26086124e-02
  3.72838165e-02  2.86008955e-01 -5.95962376e-03  2.22619202e-03
  2.03082851e-01  9.46254214e-02  2.28804878e-02  1.50239596e-02
  7.62842663e-03  2.96944980e-02  5.76956115e-02  1.98194331e-01
  1.19835060e-01  1.27950353e-01  1.41393016e-01  1.36763773e-02
  8.84279656e-02  1.50534006e-01  1.69298544e-01  7.72274570e-02
  8.09506193e-02  8.25503335e-02  1.91976304e-01 -2.40211552e-02
  2.63716262e-02  7.70531902e-02  2.21097147e-02 -1.08218412e-01
  1.64035238e-01  1.81197841e-02  6.16317853e-02 -1.08238877e-01
 -5.01879512e-02  1.38866800e-01  1.03606556e-01 -4.27875495e-02
  1.06855000e-02 -

2.8504573263784747e-07

In [44]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.8504565138680096e-07
Error r:  1.3801504176161858e-16
Error suma de pesos:  0.0


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [46]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6965	---		9481.6958
1	0.0019	0.0	0.1041	0.0005	1	9481.6958
2	0.0019	0.0	0.0028	0.0	1	9481.6958
Error of x with respect to x_ast: 0.002754207971876388
Approximate solution: [ 1.58429838e-01 -2.08662572e-02  1.58036015e-01 -9.28827511e-02
  3.16796179e-02  5.10933254e-02  8.72794066e-02  1.26086124e-02
  3.72838165e-02  2.86008955e-01 -5.95962376e-03  2.22619202e-03
  2.03082851e-01  9.46254214e-02  2.28804878e-02  1.50239596e-02
  7.62842663e-03  2.96944980e-02  5.76956115e-02  1.98194331e-01
  1.19835060e-01  1.27950353e-01  1.41393016e-01  1.36763773e-02
  8.84279656e-02  1.50534006e-01  1.69298544e-01  7.72274570e-02
  8.09506193e-02  8.25503335e-02  1.91976304e-01 -2.40211552e-02
  2.63716262e-02  7.70531902e-02  2.21097147e-02 -1.08218412e-01
  1.64035238e-01  1.81197841e-02  6.16317853e-02 -1.08238877e-01
 -5.01879512e-02  1.38866800e-01  1.03606556e-01 -4.27875495e-02
  1.06855000e-02 -

2.8504573263784747e-07

In [47]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.8504565138680096e-07
Error r:  1.3801504176161858e-16
Error suma de pesos:  0.0


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [49]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6965	---		9481.6958
1	0.0019	0.0	0.1041	0.0005	1	9481.6958
2	0.0019	0.0	0.0028	0.0	1	9481.6958
Error of x with respect to x_ast: 0.002754207971876388
Approximate solution: [ 1.58429838e-01 -2.08662572e-02  1.58036015e-01 -9.28827511e-02
  3.16796179e-02  5.10933254e-02  8.72794066e-02  1.26086124e-02
  3.72838165e-02  2.86008955e-01 -5.95962376e-03  2.22619202e-03
  2.03082851e-01  9.46254214e-02  2.28804878e-02  1.50239596e-02
  7.62842663e-03  2.96944980e-02  5.76956115e-02  1.98194331e-01
  1.19835060e-01  1.27950353e-01  1.41393016e-01  1.36763773e-02
  8.84279656e-02  1.50534006e-01  1.69298544e-01  7.72274570e-02
  8.09506193e-02  8.25503335e-02  1.91976304e-01 -2.40211552e-02
  2.63716262e-02  7.70531902e-02  2.21097147e-02 -1.08218412e-01
  1.64035238e-01  1.81197841e-02  6.16317853e-02 -1.08238877e-01
 -5.01879512e-02  1.38866800e-01  1.03606556e-01 -4.27875495e-02
  1.06855000e-02 -

2.8504573263784747e-07

In [50]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.8504565138680096e-07
Error r:  1.3801504176161858e-16
Error suma de pesos:  0.0


In [0]:
#Newton
tol=1e-14
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [52]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6965	---		9481.6958
1	0.0019	0.0	0.1041	0.0005	1	9481.6958
2	0.0019	0.0	0.0028	0.0	1	9481.6958
3	0.0019	0.0	0.0001	0.0	1	9481.6958
4	0.0019	0.0	0.0	0.0	1	9481.6958
Error of x with respect to x_ast: 3.031132220976894e-07
Approximate solution: [ 1.58463783e-01 -2.08636897e-02  1.58052223e-01 -9.29027893e-02
  3.17035768e-02  5.10936720e-02  8.71909596e-02  1.26214741e-02
  3.72847467e-02  2.86012510e-01 -5.98823128e-03  2.22637003e-03
  2.03105163e-01  9.46038156e-02  2.28670762e-02  1.49956434e-02
  7.60916104e-03  2.96818810e-02  5.76661702e-02  1.98159700e-01
  1.19888060e-01  1.27852784e-01  1.41319107e-01  1.36660757e-02
  8.83936126e-02  1.50480387e-01  1.69310059e-01  7.72153880e-02
  8.09180898e-02  8.24856614e-02  1.91986091e-01 -2.40881569e-02
  2.63690413e-02  7.69721820e-02  2.21228901e-02 -1.08217300e-01
  1.64028300e-01  1.81505545e-02  6.16482773e-02 -1.08205642e-01
 -5.02027555e

4.842364082573431e-14

In [53]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  4.8423640825731965e-14
Error r:  1.3801504176161858e-16
Error suma de pesos:  4.440892098500624e-16


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [55]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0019	0.0009	1.1003	4.6965	---		9481.6958
1	0.0019	0.0	0.1041	0.0005	1	9481.6958
2	0.0019	0.0	0.0028	0.0	1	9481.6958
3	0.0019	0.0	0.0001	0.0	1	9481.6958
4	0.0019	0.0	0.0	0.0	1	9481.6958
Error of x with respect to x_ast: 3.031132220976894e-07
Approximate solution: [ 1.58463783e-01 -2.08636897e-02  1.58052223e-01 -9.29027893e-02
  3.17035768e-02  5.10936720e-02  8.71909596e-02  1.26214741e-02
  3.72847467e-02  2.86012510e-01 -5.98823128e-03  2.22637003e-03
  2.03105163e-01  9.46038156e-02  2.28670762e-02  1.49956434e-02
  7.60916104e-03  2.96818810e-02  5.76661702e-02  1.98159700e-01
  1.19888060e-01  1.27852784e-01  1.41319107e-01  1.36660757e-02
  8.83936126e-02  1.50480387e-01  1.69310059e-01  7.72153880e-02
  8.09180898e-02  8.24856614e-02  1.91986091e-01 -2.40881569e-02
  2.63690413e-02  7.69721820e-02  2.21228901e-02 -1.08217300e-01
  1.64028300e-01  1.81505545e-02  6.16482773e-02 -1.08205642e-01
 -5.02027555e

4.842364082573431e-14

In [56]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  4.8423640825731965e-14
Error r:  1.3801504176161858e-16
Error suma de pesos:  4.440892098500624e-16


Ahora probamos con $r=0.3$

In [0]:
r=0.3
w_ast = mkv.markowitz(r,mu,S)
w_0 = utils.feasible_markowitz(r,mu)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)

In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50


In [59]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]


I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9543	---		12081.7705
1	0.0025	0.0	0.2387	0.0022	1	12081.7705
2	0.0025	0.0	0.0005	0.0	1	12081.7705
Error of x with respect to x_ast: 0.0005189696439787124
Approximate solution: [ 1.40213888e-01 -1.35552723e-02  1.10335796e-01 -4.37947478e-02
  4.33942509e-02  6.25611789e-02  6.23249533e-02  3.43253666e-02
  1.48844568e-02  2.18631993e-01 -1.85280422e-03  5.02111317e-04
  1.90513241e-01  8.94316473e-02  3.20926507e-03  1.05169150e-02
  6.46503931e-03  3.10791840e-02  5.03546798e-02  1.51305017e-01
  8.35171724e-02  1.09255225e-01  7.70926669e-02  3.71826050e-02
  7.11575919e-02  1.15434591e-01  1.14161600e-01  4.22929702e-02
  5.82207571e-02  4.24453317e-02  1.85399701e-01 -3.66975667e-02
  3.70397694e-02  7.25800932e-02 -3.11250907e-03 -8.38931299e-02
  1.36133670e-01  2.45881010e-02  4.86685562e-02 -8.05366209e-02
 -4.61056090e-02  9.43157475e-02  8.10349278e-02 -2.07811248e-02
  7.86659889e

5.670447675083701e-08

In [60]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  5.6704473535439514e-08
Error r:  0.0
Error suma de pesos:  6.661338147750935e-16


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [62]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9543	---		12081.7705
1	0.0025	0.0	0.2387	0.0022	1	12081.7705
2	0.0025	0.0	0.0005	0.0	1	12081.7705
Error of x with respect to x_ast: 0.0005189696439787124
Approximate solution: [ 1.40213888e-01 -1.35552723e-02  1.10335796e-01 -4.37947478e-02
  4.33942509e-02  6.25611789e-02  6.23249533e-02  3.43253666e-02
  1.48844568e-02  2.18631993e-01 -1.85280422e-03  5.02111317e-04
  1.90513241e-01  8.94316473e-02  3.20926507e-03  1.05169150e-02
  6.46503931e-03  3.10791840e-02  5.03546798e-02  1.51305017e-01
  8.35171724e-02  1.09255225e-01  7.70926669e-02  3.71826050e-02
  7.11575919e-02  1.15434591e-01  1.14161600e-01  4.22929702e-02
  5.82207571e-02  4.24453317e-02  1.85399701e-01 -3.66975667e-02
  3.70397694e-02  7.25800932e-02 -3.11250907e-03 -8.38931299e-02
  1.36133670e-01  2.45881010e-02  4.86685562e-02 -8.05366209e-02
 -4.61056090e-02  9.43157475e-02  8.10349278e-02 -2.07811248e-02
  7.86659889e

5.670447675083701e-08

In [63]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  5.6704473535439514e-08
Error r:  0.0
Error suma de pesos:  6.661338147750935e-16


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [65]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9543	---		12081.7705
1	0.0025	0.0	0.2387	0.0022	1	12081.7705
2	0.0025	0.0	0.0005	0.0	1	12081.7705
Error of x with respect to x_ast: 0.0005189696439787124
Approximate solution: [ 1.40213888e-01 -1.35552723e-02  1.10335796e-01 -4.37947478e-02
  4.33942509e-02  6.25611789e-02  6.23249533e-02  3.43253666e-02
  1.48844568e-02  2.18631993e-01 -1.85280422e-03  5.02111317e-04
  1.90513241e-01  8.94316473e-02  3.20926507e-03  1.05169150e-02
  6.46503931e-03  3.10791840e-02  5.03546798e-02  1.51305017e-01
  8.35171724e-02  1.09255225e-01  7.70926669e-02  3.71826050e-02
  7.11575919e-02  1.15434591e-01  1.14161600e-01  4.22929702e-02
  5.82207571e-02  4.24453317e-02  1.85399701e-01 -3.66975667e-02
  3.70397694e-02  7.25800932e-02 -3.11250907e-03 -8.38931299e-02
  1.36133670e-01  2.45881010e-02  4.86685562e-02 -8.05366209e-02
 -4.61056090e-02  9.43157475e-02  8.10349278e-02 -2.07811248e-02
  7.86659889e

5.670447675083701e-08

In [66]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  5.6704473535439514e-08
Error r:  0.0
Error suma de pesos:  6.661338147750935e-16


In [0]:
#Newton
tol=1e-14
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [68]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9543	---		12081.7705
1	0.0025	0.0	0.2387	0.0022	1	12081.7705
2	0.0025	0.0	0.0005	0.0	1	12081.7705
3	0.0025	0.0	0.0	0.0	1	12081.7705
Error of x with respect to x_ast: 1.263552029503516e-05
Approximate solution: [ 1.40208859e-01 -1.35701313e-02  1.10344514e-01 -4.37785956e-02
  4.34166503e-02  6.25598381e-02  6.23707859e-02  3.43354223e-02
  1.49777360e-02  2.18651776e-01 -1.87571027e-03  5.05670061e-04
  1.90535990e-01  8.93617177e-02  3.24189634e-03  1.05073338e-02
  6.48600212e-03  3.10749446e-02  5.03464699e-02  1.51320391e-01
  8.35518500e-02  1.09295292e-01  7.71218023e-02  3.72788939e-02
  7.11653880e-02  1.15402530e-01  1.14132258e-01  4.23118550e-02
  5.81895682e-02  4.24484140e-02  1.85352796e-01 -3.66540070e-02
  3.70840806e-02  7.25208463e-02 -3.15163365e-03 -8.39240102e-02
  1.36117737e-01  2.45592118e-02  4.86624727e-02 -8.05780496e-02
 -4.60675246e-02  9.43063435e-02  8.10042939

6.011417837751798e-12

In [69]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  6.011417837715662e-12
Error r:  1.8503717077085946e-16
Error suma de pesos:  2.2204460492503126e-16


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [71]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0025	0.0021	1.3834	14.9543	---		12081.7705
1	0.0025	0.0	0.2387	0.0022	1	12081.7705
2	0.0025	0.0	0.0005	0.0	1	12081.7705
3	0.0025	0.0	0.0	0.0	1	12081.7705
4	0.0025	0.0	0.0	0.0	1	12081.7705
Error of x with respect to x_ast: 2.8579782669796953e-07
Approximate solution: [ 1.40208825e-01 -1.35701020e-02  1.10344444e-01 -4.37786414e-02
  4.34166529e-02  6.25598493e-02  6.23709110e-02  3.43354765e-02
  1.49776709e-02  2.18651802e-01 -1.87571928e-03  5.05624952e-04
  1.90535764e-01  8.93618536e-02  3.24186281e-03  1.05075154e-02
  6.48606675e-03  3.10748092e-02  5.03466994e-02  1.51320379e-01
  8.35517225e-02  1.09295908e-01  7.71218759e-02  3.72785373e-02
  7.11654032e-02  1.15402683e-01  1.14132295e-01  4.23119000e-02
  5.81895581e-02  4.24485121e-02  1.85352719e-01 -3.66534139e-02
  3.70838697e-02  7.25209180e-02 -3.15181695e-03 -8.39240821e-02
  1.36117730e-01  2.45591067e-02  4.86624444e-02 -8.05782317e-02
 -4.60675

3.693830694909096e-14

In [72]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.69383069490896e-14
Error r:  1.8503717077085946e-16
Error suma de pesos:  2.2204460492503136e-16


Proseguimos a probar con $r=0.5$

In [0]:
r=0.5
w_ast = mkv.markowitz(r,mu,S)
w_0 = utils.feasible_markowitz(r,mu)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [75]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0319	3.348	126.523	---		4161.5799
1	0.0079	0.0001	0.4474	0.2929	1	4161.5799
2	0.0079	0.0	0.0031	0.0	1	4161.5799
Error of x with respect to x_ast: 0.0030953417065468472
Approximate solution: [ 0.17605737 -0.028133    0.20370441 -0.13944823  0.02040444  0.03971301
  0.11061157 -0.00831907  0.05926363  0.35044085 -0.00999214  0.00402387
  0.21490561  0.09919148  0.04130532  0.0192279   0.00894235  0.02825702
  0.06442817  0.24236893  0.15382738  0.14563072  0.20289951 -0.00930442
  0.1048669   0.18407827  0.2221503   0.11064296  0.10261619  0.12092284
  0.19766992 -0.01251661  0.01530187  0.08173483  0.0459347  -0.13148722
  0.1908446   0.01222174  0.07391153 -0.13464446 -0.05502942  0.1814344
  0.12511717 -0.06424143  0.01251383 -2.71010416  0.06249577  0.29287141
  0.12949049 -0.14880313]


3.91196816936474e-06

In [76]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.911952865929649e-06
Error r:  4.440892098500624e-16
Error suma de pesos:  4.440892098500624e-16


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [78]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0319	3.348	126.523	---		4161.5799
1	0.0079	0.0001	0.4474	0.2929	1	4161.5799
2	0.0079	0.0	0.0031	0.0	1	4161.5799
Error of x with respect to x_ast: 0.0030953417065468472
Approximate solution: [ 0.17605737 -0.028133    0.20370441 -0.13944823  0.02040444  0.03971301
  0.11061157 -0.00831907  0.05926363  0.35044085 -0.00999214  0.00402387
  0.21490561  0.09919148  0.04130532  0.0192279   0.00894235  0.02825702
  0.06442817  0.24236893  0.15382738  0.14563072  0.20289951 -0.00930442
  0.1048669   0.18407827  0.2221503   0.11064296  0.10261619  0.12092284
  0.19766992 -0.01251661  0.01530187  0.08173483  0.0459347  -0.13148722
  0.1908446   0.01222174  0.07391153 -0.13464446 -0.05502942  0.1814344
  0.12511717 -0.06424143  0.01251383 -2.71010416  0.06249577  0.29287141
  0.12949049 -0.14880313]


3.91196816936474e-06

In [79]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.911952865929649e-06
Error r:  4.440892098500624e-16
Error suma de pesos:  4.440892098500624e-16


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [81]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0319	3.348	126.523	---		4161.5799
1	0.0079	0.0001	0.4474	0.2929	1	4161.5799
2	0.0079	0.0	0.0031	0.0	1	4161.5799
3	0.0079	0.0	0.0001	0.0	1	4161.5799
Error of x with respect to x_ast: 8.440252455282802e-05
Approximate solution: [ 0.17592781 -0.02784114  0.20369593 -0.13990183  0.02049809  0.04012451
  0.11094177 -0.00815018  0.05862599  0.3504588  -0.00992193  0.00387246
  0.2151303   0.09961973  0.04164509  0.01929082  0.00868628  0.02835077
  0.06467366  0.24297664  0.15465774  0.14561731  0.20274366 -0.0089252
  0.10487644  0.1840411   0.22210071  0.11060793  0.10266521  0.12079228
  0.19833116 -0.01205976  0.01611629  0.08122996  0.04630415 -0.13146072
  0.19072997  0.01201844  0.07407202 -0.13463793 -0.05415703  0.18144813
  0.12517954 -0.06387992  0.01330822 -2.70779794  0.05451904  0.2922097
  0.12956636 -0.14892044]


3.6350116555457286e-10

In [82]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.6350116542243975e-10
Error r:  6.661338147750935e-16
Error suma de pesos:  1.1102230246251554e-15


In [0]:
#Newton
tol=1e-14
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [84]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0319	3.348	126.523	---		4161.5799
1	0.0079	0.0001	0.4474	0.2929	1	4161.5799
2	0.0079	0.0	0.0031	0.0	1	4161.5799
3	0.0079	0.0	0.0001	0.0	1	4161.5799
4	0.0079	0.0	0.0	0.0	1	4161.5799
Error of x with respect to x_ast: 4.650994220985898e-07
Approximate solution: [ 0.17592898 -0.02784175  0.2036962  -0.13990185  0.02049718  0.04012363
  0.11093726 -0.00815301  0.0586269   0.3504591  -0.00992289  0.00387263
  0.21513084  0.09961907  0.04164334  0.0192896   0.00868362  0.02834922
  0.06466904  0.24297272  0.15465277  0.14560683  0.20273914 -0.00892477
  0.10487635  0.18404044  0.22210081  0.11060885  0.10266334  0.12079067
  0.19833255 -0.01206673  0.01611771  0.08123077  0.04630413 -0.13145965
  0.1907314   0.01201928  0.07407236 -0.13463794 -0.05415913  0.18144706
  0.12517924 -0.0638816   0.01330772 -2.70760142  0.05437719  0.29221037
  0.12956553 -0.1489231 ]


1.1165550098866728e-13

In [85]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  1.1165550098865483e-13
Error r:  4.440892098500624e-16
Error suma de pesos:  8.881784197001244e-16


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [87]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0079	0.0319	3.348	126.523	---		4161.5799
1	0.0079	0.0001	0.4474	0.2929	1	4161.5799
2	0.0079	0.0	0.0031	0.0	1	4161.5799
3	0.0079	0.0	0.0001	0.0	1	4161.5799
4	0.0079	0.0	0.0	0.0	1	4161.5799
Error of x with respect to x_ast: 4.650994220985898e-07
Approximate solution: [ 0.17592898 -0.02784175  0.2036962  -0.13990185  0.02049718  0.04012363
  0.11093726 -0.00815301  0.0586269   0.3504591  -0.00992289  0.00387263
  0.21513084  0.09961907  0.04164334  0.0192896   0.00868362  0.02834922
  0.06466904  0.24297272  0.15465277  0.14560683  0.20273914 -0.00892477
  0.10487635  0.18404044  0.22210081  0.11060885  0.10266334  0.12079067
  0.19833255 -0.01206673  0.01611771  0.08123077  0.04630413 -0.13145965
  0.1907314   0.01201928  0.07407236 -0.13463794 -0.05415913  0.18144706
  0.12517924 -0.0638816   0.01330772 -2.70760142  0.05437719  0.29221037
  0.12956553 -0.1489231 ]


1.1165550098866728e-13

In [88]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  1.1165550098865483e-13
Error r:  4.440892098500624e-16
Error suma de pesos:  8.881784197001244e-16


## Método de Newton usando funciones simbólicas


In [89]:
lines = inspect.getsource(opt.gfo_cp_mark)
print(lines)

def gfo_cp_mark(Sigma,x):
    '''
    gradiente de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma y vector x
    output: producto matriz-vector Sigma*x
    '''
    first_block = Sigma@x
    return first_block



In [90]:
lines = inspect.getsource(opt.Hfo_cp_mark)
print(lines)

def Hfo_cp_mark(Sigma):
    '''
    Hessiana de la función objetivo 1/2*x.t*Sigma*x
    input: matriz Sigma
    output: matriz Sigma
    '''
    first_block = Sigma 
    return first_block



In [0]:
r=max(mu).item()
w_ast = mkv.markowitz(r,mu,S)
w_0 = utils.feasible_markowitz(r,mu)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)

In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [93]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.001	0.0004	1.1003	4.6965	---		10080.7626
1	0.001	0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 3.897597330727876e-12
Approximate solution: [ 1.58463748e-01 -2.08636961e-02  1.58052232e-01 -9.29027844e-02
  3.17035790e-02  5.10937141e-02  8.71909946e-02  1.26214911e-02
  3.72848212e-02  2.86012499e-01 -5.98823678e-03  2.22635537e-03
  2.03105240e-01  9.46037995e-02  2.28671081e-02  1.49956262e-02
  7.60913712e-03  2.96819117e-02  5.76661968e-02  1.98159753e-01
  1.19888132e-01  1.27852786e-01  1.41319157e-01  1.36660783e-02
  8.83935972e-02  1.50480332e-01  1.69310094e-01  7.72153681e-02
  8.09181082e-02  8.24857024e-02  1.91986129e-01 -2.40882038e-02
  2.63691159e-02  7.69721727e-02  2.21228946e-02 -1.08217307e-01
  1.64028304e-01  1.81505775e-02  6.16482988e-02 -1.08205644e-01
 -5.02026895e-02  1.38840045e-01  1.03580114e-01 -4.28041951e-02
  1.06734849e-02 -2.33324982e+00  1.89509763e-01  2.6319627

2.1489781431538896e-15

In [94]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.148978143153885e-15
Error r:  1.380150417616186e-16
Error suma de pesos:  4.440892098500624e-16


In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [96]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.001	0.0004	1.1003	4.6965	---		10080.7626
1	0.001	0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 3.897597330727876e-12
Approximate solution: [ 1.58463748e-01 -2.08636961e-02  1.58052232e-01 -9.29027844e-02
  3.17035790e-02  5.10937141e-02  8.71909946e-02  1.26214911e-02
  3.72848212e-02  2.86012499e-01 -5.98823678e-03  2.22635537e-03
  2.03105240e-01  9.46037995e-02  2.28671081e-02  1.49956262e-02
  7.60913712e-03  2.96819117e-02  5.76661968e-02  1.98159753e-01
  1.19888132e-01  1.27852786e-01  1.41319157e-01  1.36660783e-02
  8.83935972e-02  1.50480332e-01  1.69310094e-01  7.72153681e-02
  8.09181082e-02  8.24857024e-02  1.91986129e-01 -2.40882038e-02
  2.63691159e-02  7.69721727e-02  2.21228946e-02 -1.08217307e-01
  1.64028304e-01  1.81505775e-02  6.16482988e-02 -1.08205644e-01
 -5.02026895e-02  1.38840045e-01  1.03580114e-01 -4.28041951e-02
  1.06734849e-02 -2.33324982e+00  1.89509763e-01  2.6319627

2.1489781431538896e-15

In [97]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.148978143153885e-15
Error r:  1.380150417616186e-16
Error suma de pesos:  4.440892098500624e-16


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [99]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.001	0.0004	1.1003	4.6965	---		10080.7626
1	0.001	0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 3.897597330727876e-12
Approximate solution: [ 1.58463748e-01 -2.08636961e-02  1.58052232e-01 -9.29027844e-02
  3.17035790e-02  5.10937141e-02  8.71909946e-02  1.26214911e-02
  3.72848212e-02  2.86012499e-01 -5.98823678e-03  2.22635537e-03
  2.03105240e-01  9.46037995e-02  2.28671081e-02  1.49956262e-02
  7.60913712e-03  2.96819117e-02  5.76661968e-02  1.98159753e-01
  1.19888132e-01  1.27852786e-01  1.41319157e-01  1.36660783e-02
  8.83935972e-02  1.50480332e-01  1.69310094e-01  7.72153681e-02
  8.09181082e-02  8.24857024e-02  1.91986129e-01 -2.40882038e-02
  2.63691159e-02  7.69721727e-02  2.21228946e-02 -1.08217307e-01
  1.64028304e-01  1.81505775e-02  6.16482988e-02 -1.08205644e-01
 -5.02026895e-02  1.38840045e-01  1.03580114e-01 -4.28041951e-02
  1.06734849e-02 -2.33324982e+00  1.89509763e-01  2.6319627

2.1489781431538896e-15

In [100]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.148978143153885e-15
Error r:  1.380150417616186e-16
Error suma de pesos:  4.440892098500624e-16


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [102]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.001	0.0004	1.1003	4.6965	---		10080.7626
1	0.001	0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 3.897597330727876e-12
Approximate solution: [ 1.58463748e-01 -2.08636961e-02  1.58052232e-01 -9.29027844e-02
  3.17035790e-02  5.10937141e-02  8.71909946e-02  1.26214911e-02
  3.72848212e-02  2.86012499e-01 -5.98823678e-03  2.22635537e-03
  2.03105240e-01  9.46037995e-02  2.28671081e-02  1.49956262e-02
  7.60913712e-03  2.96819117e-02  5.76661968e-02  1.98159753e-01
  1.19888132e-01  1.27852786e-01  1.41319157e-01  1.36660783e-02
  8.83935972e-02  1.50480332e-01  1.69310094e-01  7.72153681e-02
  8.09181082e-02  8.24857024e-02  1.91986129e-01 -2.40882038e-02
  2.63691159e-02  7.69721727e-02  2.21228946e-02 -1.08217307e-01
  1.64028304e-01  1.81505775e-02  6.16482988e-02 -1.08205644e-01
 -5.02026895e-02  1.38840045e-01  1.03580114e-01 -4.28041951e-02
  1.06734849e-02 -2.33324982e+00  1.89509763e-01  2.6319627

2.1489781431538896e-15

In [103]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.148978143153885e-15
Error r:  1.380150417616186e-16
Error suma de pesos:  4.440892098500624e-16


$r=0.3$

In [0]:
r = 0.3
w_ast = mkv.markowitz(r,mu,S)
w_0 = utils.feasible_markowitz(r,mu)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)

In [105]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0012	0.001	1.3834	10.757	---		10080.7626
1	0.0012	-0.0	0.0	0.2631	1	10080.7626
Error of x with respect to x_ast: 5.350769330348929e-12
Approximate solution: [ 1.40208822e-01 -1.35700942e-02  1.10344420e-01 -4.37786342e-02
  4.34166657e-02  6.25598563e-02  6.23709322e-02  3.43354423e-02
  1.49776433e-02  2.18651815e-01 -1.87568245e-03  5.05631159e-04
  1.90535779e-01  8.93618800e-02  3.24187501e-03  1.05075495e-02
  6.48607041e-03  3.10748292e-02  5.03467102e-02  1.51320408e-01
  8.35516867e-02  1.09295893e-01  7.71219290e-02  3.72785210e-02
  7.11654751e-02  1.15402707e-01  1.14132288e-01  4.23118942e-02
  5.81895496e-02  4.24485048e-02  1.85352738e-01 -3.66533431e-02
  3.70839365e-02  7.25209744e-02 -3.15183482e-03 -8.39240713e-02
  1.36117732e-01  2.45591018e-02  4.86624414e-02 -8.05782046e-02
 -4.60674991e-02  9.43064381e-02  8.10042750e-02 -2.07737144e-02
  7.92009810e-03 -1.94196978e+00  3.30752412e-01  2.32

In [106]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.5273578438851775e-15
Error r:  1.8503717077085946e-16
Error suma de pesos:  2.2204460492503083e-15


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [108]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0012	0.001	1.3834	14.9543	---		10080.7626
1	0.0012	-0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 5.350769330348929e-12
Approximate solution: [ 1.40208822e-01 -1.35700942e-02  1.10344420e-01 -4.37786342e-02
  4.34166657e-02  6.25598563e-02  6.23709322e-02  3.43354423e-02
  1.49776433e-02  2.18651815e-01 -1.87568245e-03  5.05631159e-04
  1.90535779e-01  8.93618800e-02  3.24187501e-03  1.05075495e-02
  6.48607041e-03  3.10748292e-02  5.03467102e-02  1.51320408e-01
  8.35516867e-02  1.09295893e-01  7.71219290e-02  3.72785210e-02
  7.11654751e-02  1.15402707e-01  1.14132288e-01  4.23118942e-02
  5.81895496e-02  4.24485048e-02  1.85352738e-01 -3.66533431e-02
  3.70839365e-02  7.25209744e-02 -3.15183482e-03 -8.39240713e-02
  1.36117732e-01  2.45591018e-02  4.86624414e-02 -8.05782046e-02
 -4.60674991e-02  9.43064381e-02  8.10042750e-02 -2.07737144e-02
  7.92009810e-03 -1.94196978e+00  3.30752412e-01  2.3287

2.5273578438851712e-15

In [109]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.5273578438851775e-15
Error r:  1.8503717077085946e-16
Error suma de pesos:  2.2204460492503083e-15


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [111]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.0012	0.001	1.3834	14.9543	---		10080.7626
1	0.0012	-0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 5.350769330348929e-12
Approximate solution: [ 1.40208822e-01 -1.35700942e-02  1.10344420e-01 -4.37786342e-02
  4.34166657e-02  6.25598563e-02  6.23709322e-02  3.43354423e-02
  1.49776433e-02  2.18651815e-01 -1.87568245e-03  5.05631159e-04
  1.90535779e-01  8.93618800e-02  3.24187501e-03  1.05075495e-02
  6.48607041e-03  3.10748292e-02  5.03467102e-02  1.51320408e-01
  8.35516867e-02  1.09295893e-01  7.71219290e-02  3.72785210e-02
  7.11654751e-02  1.15402707e-01  1.14132288e-01  4.23118942e-02
  5.81895496e-02  4.24485048e-02  1.85352738e-01 -3.66533431e-02
  3.70839365e-02  7.25209744e-02 -3.15183482e-03 -8.39240713e-02
  1.36117732e-01  2.45591018e-02  4.86624414e-02 -8.05782046e-02
 -4.60674991e-02  9.43064381e-02  8.10042750e-02 -2.07737144e-02
  7.92009810e-03 -1.94196978e+00  3.30752412e-01  2.3287

2.5273578438851712e-15

In [112]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  2.5273578438851775e-15
Error r:  1.8503717077085946e-16
Error suma de pesos:  2.2204460492503083e-15


$r=0.5$

In [0]:
r = 0.5
w_ast = mkv.markowitz(r,mu,S)
n = mu.shape[0]
#Matriz A de restricciones
A = cp.concatenate((mu,cp.ones(n))).reshape(2,n)
#Vector b
b = cp.array([r,1])
#Punto inicial
w_0 = utils.feasible_markowitz(r,mu)

In [0]:
#Newton
tol=1e-8
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=50

In [115]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.004	0.0159	3.348	126.523	---		10080.7626
1	0.004	0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 3.2754229245508606e-12
Approximate solution: [ 0.17592894 -0.02784177  0.20369615 -0.13990177  0.02049721  0.04012361
  0.11093731 -0.00815309  0.05862697  0.35045908 -0.00992288  0.00387264
  0.21513093  0.09961895  0.04164333  0.01928954  0.00868362  0.02834925
  0.06466903  0.24297278  0.15465262  0.14560689  0.20273914 -0.00892486
  0.10487641  0.18404046  0.22210085  0.11060888  0.1026634   0.12079085
  0.19833255 -0.01206665  0.01611783  0.08123081  0.04630421 -0.13145959
  0.19073143  0.01201929  0.07407237 -0.13463789 -0.05415899  0.18144708
  0.1251793  -0.06388161  0.01330776 -2.70760264  0.05437743  0.29221027
  0.1295655  -0.14892295]


3.6643697235663006e-15

In [116]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.664369723566287e-15
Error r:  4.440892098500624e-16
Error suma de pesos:  2.1094237467878018e-15


In [0]:
#Newton
tol=1e-10
tol_backtracking=1e-14
p_ast=fo(w_ast)
maxiter=500

In [118]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.004	0.0159	3.348	126.523	---		10080.7626
1	0.004	0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 3.2754229245508606e-12
Approximate solution: [ 0.17592894 -0.02784177  0.20369615 -0.13990177  0.02049721  0.04012361
  0.11093731 -0.00815309  0.05862697  0.35045908 -0.00992288  0.00387264
  0.21513093  0.09961895  0.04164333  0.01928954  0.00868362  0.02834925
  0.06466903  0.24297278  0.15465262  0.14560689  0.20273914 -0.00892486
  0.10487641  0.18404046  0.22210085  0.11060888  0.1026634   0.12079085
  0.19833255 -0.01206665  0.01611783  0.08123081  0.04630421 -0.13145959
  0.19073143  0.01201929  0.07407237 -0.13463789 -0.05415899  0.18144708
  0.1251793  -0.06388161  0.01330776 -2.70760264  0.05437743  0.29221027
  0.1295655  -0.14892295]


3.6643697235663006e-15

In [119]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.664369723566287e-15
Error r:  4.440892098500624e-16
Error suma de pesos:  2.1094237467878018e-15


In [0]:
#Newton
tol=1e-16
tol_backtracking=1e-15
p_ast=fo(w_ast)
maxiter=500

In [121]:
[w,total_of_iterations,Err_plot,x_plot]=opt.Newtons_method_feasible_init_point(fo,A, w_0,tol, tol_backtracking, w_ast, p_ast, maxiter, opt.gfo_cp_mark, opt.Hfo_cp_mark,S)
#Error
Err_plot[-1]

I	Normgf 	Newton Decrement	Error x_ast	Error p_ast	line search	CondHf
0	0.004	0.0159	3.348	126.523	---		10080.7626
1	0.004	0.0	0.0	0.0	1	10080.7626
Error of x with respect to x_ast: 3.2754229245508606e-12
Approximate solution: [ 0.17592894 -0.02784177  0.20369615 -0.13990177  0.02049721  0.04012361
  0.11093731 -0.00815309  0.05862697  0.35045908 -0.00992288  0.00387264
  0.21513093  0.09961895  0.04164333  0.01928954  0.00868362  0.02834925
  0.06466903  0.24297278  0.15465262  0.14560689  0.20273914 -0.00892486
  0.10487641  0.18404046  0.22210085  0.11060888  0.1026634   0.12079085
  0.19833255 -0.01206665  0.01611783  0.08123081  0.04630421 -0.13145959
  0.19073143  0.01201929  0.07407237 -0.13463789 -0.05415899  0.18144708
  0.1251793  -0.06388161  0.01330776 -2.70760264  0.05437743  0.29221027
  0.1295655  -0.14892295]


3.6643697235663006e-15

In [122]:
print('Error w: ', utils.compute_error(w@S@w,w_ast@S@w_ast))
print('Error r: ', utils.compute_error(w@mu, r))
print('Error suma de pesos: ', abs(((sum(w)-1)/(sum(w)))))

Error w:  3.664369723566287e-15
Error r:  4.440892098500624e-16
Error suma de pesos:  2.1094237467878018e-15


# Hallazgos

El código funciona de manera correcta para distintos niveles de tolerancia, iteraciones y rendimientos r.

Las funciones generan unos valores con una exactitud de hasta 13 dígitos correctos en el caso de Método de Newton usando diferencias fifnitas y de hasta 15 dígotos correctos en el caso de Método de Newton usando funciones simbólicas. 

